In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
import re

/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-01 20:56:29,395] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/lyus4/anaconda3/envs/rag_env/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/lyus4/anaconda3/envs/rag_env/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# 1.加载英文PDF
loader = PyMuPDFLoader("documents/Fundamentals of Wireless Communication.pdf", extract_images=True)
documents = loader.load()


# 2. 定义多条清洗正则
clean_patterns = [
    # a) 单独一行的纯数字（页码）
    re.compile(r"^\s*\d+\s*$", re.MULTILINE),
    # b) 常见英文页眉页脚
    re.compile(r"Page\s+\d+\s+of\s+\d+", re.IGNORECASE),
    # c) 常见中文页眉页脚
    re.compile(r"第\s*\d+\s*页[，,]\s*共\s*\d+\s*页"),
    # d) 章节号开头，如 "1. ", "1.1 ", "2.3.4 "
    re.compile(r"^\s*\d+(\.\d+)*\s+", re.MULTILINE),
]

# 3. 遍历每个 Document，依次应用清洗
for doc in documents:
    text = doc.page_content

    # 应用所有清洗规则
    for pat in clean_patterns:
        text = pat.sub("", text)

    # 合并多余空行
    text = re.sub(r"\n{2,}", "\n\n", text)

    doc.page_content = text



# 3.切割文本
CHUNK_SIZE = 1000
OVERLAP_SIZE = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE,
    separators=[
        "\n## ",  # 捕获章节标题
        "\n### ",
        "\n#### ",
        "\n\n",
        "\n",
        "。 ",
        "! ",
        "? ",
        ", ",
        " "
    ]
)

chunks = text_splitter.split_documents(documents)


In [3]:
print(f"原始文档数量: {len(documents)}")
print(f"切割后的文本块数量: {len(chunks)}")

原始文档数量: 644
切割后的文本块数量: 2207


In [4]:
print(chunks[105].page_content)  # 打印第11个文本块的前500个字符
print("--------------------")
print(chunks[106].page_content)  # 打印第12个文本块的前500个字符

(2.8)
where λ := c/f is the wavelength of the transmitted sinusoid.
The constructive and destructive interference pattern also depends on the frequency
f: for a ﬁxed r, if f changes by

µ2d −r
c
−r
c
¶−1
,
(2.9)
we move from a peak to a valley. The quantity
Td := 2d −r
c
−r
c
(2.10)
is called the delay spread of the channel: it is the diﬀerence between the propagation
delays along the two signal paths. Thus, the constructive and destructive interference
--------------------
Tse and Viswanath: Fundamentals of Wireless Communications

pattern changes signiﬁcantly if the frequency changes by an amount of the order of
1/Td. This parameter is called the coherence bandwidth.
Reﬂecting wall, moving antenna
Suppose the receive antenna is now moving at a velocity v (Figure 2.4). As it moves
through the pattern of constructive and destructive interference created by the two
waves, the strength of the received signal increases and decreases. This is the phe-
nomenon of multipath fading. The time 

In [5]:
# 1.加载本地英文嵌入模型
embed_model = HuggingFaceEmbeddings(
	model_name='../all-MiniLM-L6-v2',
	model_kwargs={'device': 'cpu'},
	encode_kwargs={'normalize_embeddings': True}  
)

# 2.构建本地FAISS库
db = FAISS.from_documents(chunks, embed_model)
db.save_local('db/faisee_en')


/tmp/ipykernel_3712572/1588966758.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
No sentence-transformers model found with name ../all-MiniLM-L6-v2. Creating a new one with mean pooling.


In [6]:
# 1.加载本地deepseek-7b模型
model_dir = "../deepseek-llm-7b-chat"
tok   = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir,
                                             trust_remote_code=True,
                                             torch_dtype="auto",
                                             device_map="auto")
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tok,
                max_new_tokens=256,    # 足够即可，不要太大
                do_sample=True,
				temperature=0.1)

llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.57s/it]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
Device set to use cuda:0
/tmp/ipykernel_3712572/1055922262.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [17]:
question =  "什么是ofdm?"
retriever = db.as_retriever(
    search_type="mmr",  # 最大边际相关性，提升多样性
    search_kwargs={"k": 2, "fetch_k": 20}
)
docs = retriever.invoke(question)
print(f"检索到的内容数:", {len(docs)})
print(docs)


检索到的内容数: {2}
[Document(id='86c5713d-87f9-454e-b1d5-125f4feb2acd', metadata={'producer': 'dvipdfm 0.13.2c, Copyright © 1998, by Mark A. Wicks', 'creator': 'TeX output 2004.12.09:1440', 'creationdate': '2004-12-09T14:45:01-08:00', 'source': 'documents/Fundamentals of Wireless Communication.pdf', 'file_path': 'documents/Fundamentals of Wireless Communication.pdf', 'total_pages': 644, 'format': 'PDF 1.2', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-05-18T19:26:23+08:00', 'trapped': '', 'modDate': "D:20220518192623+08'00'", 'creationDate': "D:20041209144501-08'00'", 'page': 211}, page_content='over the ith OFDM block can be written as:\n˜yn[i] = ˜hn ˜dn[i] + ˜wn[i]\nn = 0, 1, . . . , Nc −1.\n(5.33)\nHere,\n˜d[i]\n:=\n[ ˜d0[i], . . . ˜dNc−1[i]]t,\n(5.34)\n˜w[i]\n:=\n[ ˜w0[i], . . . ˜wNc−1[i]]t,\n(5.35)'), Document(id='a4c1ef0b-2bcf-48d8-8229-0eaaf1e2a345', metadata={'producer': 'dvipdfm 0.13.2c, Copyright © 1998, by Mark A. Wicks', 'creator': 'TeX output 2004.1

In [8]:
# for i,doc in enumerate(docs):
# 	print(f"检索到的第{i}个内容：\n {doc.page_content}", end = "\n--------------------------\n")

In [9]:
from langchain_core.runnables import RunnableLambda

def combine_docs(docs):
	return "\n\n".join([doc.page_content for doc in docs])

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

# retrieval_chain.invoke("numerology是什么?")


In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# … 省略切分/FAISS/LLM 加载部分 …

# 1️⃣ 定义 PromptTemplate 时，务必写上 input_variables
template = """<|System|>
你是一名5G通信专家，请严格按以下要求回答：
1. 仅使用提供的上下文
2. 用三句话回答，格式：
   [定义]...[技术特性]...[应用场景]
3. 最后用"谢谢你的提问！"结尾

<|Context|>
{context}

<|Question|>
{input}

<|Response|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "input"],   # ← 这一行必须加
)


# 2️⃣ 把 retriever 和输入包装成并行 runnable
qa_chain = (
    RunnableParallel({
        "input": RunnablePassthrough(),
        "context": RunnableLambda(lambda x: x["input"]) | retriever | combiner
    })
    | prompt
    | llm
    | StrOutputParser()
)

# # 3️⃣ 调用时也要传 dict，而不是直接传字符串
# result = qa_chain.invoke({"input": question_1})
# print("回答：", result)
# def clean_response(text: str) -> str:
#     # 1) 去掉所有上下文标签
#     text = text.replace("<|System|>", "")
#     text = re.sub(r"<\|Context\|>.*<\|Question\|>", "", text, flags=re.DOTALL)
#     text = text.replace("<|Question|>", "")
#     text = text.replace("<|Response|>", "")
#     # 2) 删除多余空行
#     text = re.sub(r"\n{2,}", "\n", text).strip()
#     return text

# qa_chain = qa_chain | RunnableLambda(clean_response)


In [24]:
questions = [
    "什么是OFDM?",
    "什么是频率选择性衰落",
    "什么是MIMO？"
]

for q in questions:
    print(f"\n问题：{q}")
    result = qa_chain.invoke({"input": q})  # ✅ 传 dict 而不是字符串
    print("答案：", result)
    print("-"*50)


问题：什么是OFDM?
答案： <|System|>
你是一名5G通信专家，请严格按以下要求回答：
1. 仅使用提供的上下文
2. 用三句话回答，格式：
   [定义]...[技术特性]...[应用场景]
3. 最后用"谢谢你的提问！"结尾

<|Context|>
over the ith OFDM block can be written as:
˜yn[i] = ˜hn ˜dn[i] + ˜wn[i]
n = 0, 1, . . . , Nc −1.
(5.33)
Here,
˜d[i]
:=
[ ˜d0[i], . . . ˜dNc−1[i]]t,
(5.34)
˜w[i]
:=
[ ˜w0[i], . . . ˜wNc−1[i]]t,
(5.35)

rate LDPC (low density parity check) code combined with a simple modulation
scheme (such as QPSK or 16-QAM) is used to convert the raw information bits
into the 672 OFDM symbols.
The diﬀerent levels of granularity of the traﬃc channels are ideally suited to
carry bursty traﬃc. Indeed, Flash-OFDM is designed to act in a data network
where it harnessing the statistical multiplexing gains of the user’s bursty data
traﬃc by its packet-switching operation.
The mobiles are in three diﬀerent states in the network. When they are
inactive, they go to a “sleep” mode monitoring the base station signal every once
in a while: this mode saves power by turning oﬀmost of 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


答案： <|System|>
你是一名5G通信专家，请严格按以下要求回答：
1. 仅使用提供的上下文
2. 用三句话回答，格式：
   [定义]...[技术特性]...[应用场景]
3. 最后用"谢谢你的提问！"结尾

<|Context|>
X
k=1
HkKxkH∗
k
!
.
(10.24)

Exercise B.4. Consider a continuous real random variable x with density fx(·) non-
zero on the entire real line. Suppose the second moment of x is ﬁxed to be P. Show
that among all random variables with the constraints as those on x, the Gaussian
random variable has the maximum diﬀerential entropy. Hint: The diﬀerential entropy
is a concave function of the density function and ﬁxing the second moment corresponds
to a linear constraint on the density function. So, you can use the classical Lagrangian
techniques to solve this problem.
Exercise B.5. Suppose x is now a nonnegative random variable with density non-zero
for all nonnegative real numbers. Further suppose that the mean of x is ﬁxed. Show
that among all random variables of this form, the exponential random variable has the
maximum diﬀerential entropy.
Exercise B.6. In this exercis

In [12]:
for q in questions:
    print(f"\n问题：{q}")
    result = llm.invoke(q)  # 直接传入问题字符串
    print("答案：", result)
    print("-"*50)


问题：什么是OFDM?
答案： 什么是OFDM?
OFDM（正交频分复用）是一种无线通信技术，它将高速数据流分割成多个较小的子载波，并将它们分配到不同的频率上。每个子载波都使用正交频分复用（OFDM）技术进行调制，以减少多径衰落的影响，提高通信系统的性能。OFDM技术广泛应用于无线通信系统中，如Wi-Fi、WiMAX、LTE和5G等。
--------------------------------------------------

问题：什么是频率选择性衰落
答案： 什么是频率选择性衰落信道？
频率选择性衰落信道是指在传输过程中，信号的衰落特性与接收端的频率有关。这种信道通常由多径传播引起，即信号在传输过程中会经过多个路径，每个路径的传播时间可能不同，从而导致信号的衰落。在频率选择性衰落信道中，信号的衰落特性与接收端的频率有关，即在某个频率上衰落，而在其他频率上可能不衰落。这种信道在无线通信中很常见，例如在移动通信中，由于用户的位置变化，信号的衰落特性也会发生变化。
--------------------------------------------------

问题：什么是MIMO？
答案： 什么是MIMO？
什么是MIMO？
MIMO是“多输入多输出”的缩写，是一种无线通信技术，允许无线设备同时发送和接收多个数据流。MIMO技术通过使用多个天线来增加无线信号的容量和速度。当数据通过多个天线发送时，可以同时发送多个数据流，从而提高数据传输速度和减少延迟。MIMO技术广泛应用于Wi-Fi、蜂窝网络和卫星通信等领域。
--------------------------------------------------
